In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import json
sys.path.append("..")
from psflearning.psflearninglib import psflearninglib
with open('datadir.json','r') as file:
    maindatadir = json.load(file)['main_data_dir']

In [ ]:
#load parameters
paramfile = 'params_default.json'
L = psflearninglib()
L.getparam(paramfile)

In [ ]:
# load data

L.param['datapath'] = maindatadir + r'bead data\01-04-2022 bead\40nm_top/'
L.param['keyword'] = 'bead'
L.param['subfolder'] = 'bead'
L.param['format'] = '.mat'
L.param['channeltype'] = 'single'
L.param['gain'] = 1/2.27
L.param['ccd_offset'] = 100
images = L.load_data()


In [ ]:
# setup PSF type
L.param['PSFtype'] = 'voxel'
L.getpsfclass()

In [ ]:
# crop bead
L.param['pixelsize_x'] =  0.129
L.param['pixelsize_y'] = 0.129
L.param['pixelsize_z'] = 0.05
L.param['roi_size'] = [21,21]
L.param['plotall'] = True
L.param['max_bead_number'] = 20
L.param['bead_radius'] = 0.0
L.param['FOV']['z_start'] = 0
dataobj = L.prep_data(images)

In [ ]:
# learn PSF
L.param['iteration'] = 100
L.param['vary_photon'] = False
L.param['estimate_drift'] = False
psfobj,fitter = L.learn_psf(dataobj,time=0)

In [ ]:
# save as .h5 file
L.param['savename'] = L.param['datapath'] + 'psfmodel_test'
resfile, res_dict, loc_dict, rois_dict = L.save_result(psfobj,dataobj,fitter)

In [ ]:
# show estimated bead positions, photon and background
cor = rois_dict['cor']
ref_pos1 = res_dict['pos']
fig = plt.figure(figsize=[16,8])
ax = fig.add_subplot(2,4,1)
plt.plot(ref_pos1[:,1]-cor[:,0])
plt.title('y')
ax = fig.add_subplot(2,4,2)
plt.plot(ref_pos1[:,2]-cor[:,1])
plt.title('x')
ax = fig.add_subplot(2,4,3)
plt.plot(ref_pos1[:,0])
plt.title('z')
ax = fig.add_subplot(2,4,5)
plt.plot(res_dict['intensity'].transpose())
plt.title('phton')
ax = fig.add_subplot(2,4,6)
plt.plot(res_dict['bg'])
plt.title('background')

In [ ]:
# compare PSF model and data
psf_data = rois_dict['psf_data']
psf_fit = rois_dict['psf_fit']

ind1 = 0
im1 = psf_data[ind1]
im2 = psf_fit[ind1]
Nz = im1.shape[0]
zind = range(0,Nz,4)
fig = plt.figure(figsize=[3*len(zind),6])
for i,id in enumerate(zind):
    ax = fig.add_subplot(2,len(zind),i+1)
    plt.imshow(im1[id],cmap='twilight')
    plt.axis('off')
    ax = fig.add_subplot(2,len(zind),i+1+len(zind))
    plt.imshow(im2[id],cmap='twilight')
    plt.axis('off')

In [ ]:
# show localization result
Nz = loc_dict['loc']['z'].shape[1]
fig = plt.figure(figsize=[16,4])
ax = fig.add_subplot(1,3,1)
plt.plot(loc_dict['loc']['x'].transpose()*L.param['pixelsize_x']*1e3,'k',alpha=0.1)
plt.plot(loc_dict['loc']['x'][0]*0.0,'r')
ax.set_ylabel('x bias (nm)')
ax = fig.add_subplot(1,3,2)
plt.plot(loc_dict['loc']['y'].transpose()*L.param['pixelsize_y']*1e3,'k',alpha=0.1)
plt.plot(loc_dict['loc']['x'][0]*0.0,'r')
ax.set_ylabel('y bias (nm)')
ax = fig.add_subplot(1,3,3)
plt.plot(np.transpose(loc_dict['loc']['z']-np.linspace(0,Nz-1,Nz))*L.param['pixelsize_z']*1e3,'k',alpha=0.1)
plt.plot(loc_dict['loc']['x'][0]*0.0,'r')
ax.set_ylabel('z bias (nm)')
ax.set_ylim([-40,40])